In [1]:
import yfinance as yf
import pandas as pd
from pandas_datareader import data
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta,date
from plotly.subplots import make_subplots
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist

yf.pdr_override()

In [2]:
def closest_support_resistance(close,lst):
    lst = np.asarray(lst)
    idx = (np.abs(lst-close)).argmin()
    return lst[idx]

def get_support_resist(price,last_5):
    K = 6
    kmeans = KMeans(n_clusters = K).fit(price)
    clusters = kmeans.predict(price)

    min_max_values = []

    for i in range(K):
        min_max_values.append([np.inf,-np.inf])

    for i in range(len(price)):
        cluster = clusters[i]

        # Min
        if price[i] < min_max_values[cluster][0]:
            min_max_values[cluster][0] = price[i]

        # Max    
        if price[i] > min_max_values[cluster][1]:
            min_max_values[cluster][1] = price[i]

    # List to contain
    support_resist = []

    # Sort cluster
    s = sorted(min_max_values, key = lambda x: x[0])

    # Get cluster average
    for i, (_min, _max) in enumerate(s):
        # min for first cluster
        if i == 0:
            support_resist.append(_min)

        # max for last cluster
        if i == len(min_max_values) - 1:
            support_resist.append(_max)

        # Append average
        else:
            support_resist.append(sum([_max, s[i+1][0]])/2)
            
    #30 day standard / upper band / lower band

    # Calculate and define moving average of 30 periods
    avg_30 = last_5['Close'].rolling(window=30).mean()

    # 30 days standard deviation
    std_30 = last_5['Close'].rolling(window=30).std()

    # Upper band
    upper = avg_30 + (std_30*2)

    # Lower band
    lower = avg_30 - (std_30*2)
    
    last_5['upper_band'] = upper
    last_5['lower_band'] = lower
    
    # Calculate and define moving average of 30 periods
    avg_30 = last_5['Close'].rolling(window=30, min_periods=1).mean()
    last_5['avg_30'] = avg_30
    
    
    price_action = last_5.copy()
    
    # Buy Sell Signal based on Bollinger Band Only
    price_action['signal'] = np.where(price_action['Close']>price_action['upper_band'],'Sell',
                                 np.where(price_action['Close']<price_action['lower_band'],'Buy','Hold'))
    

    buy = price_action.loc[price_action['signal']=='Buy']
    buy = buy[['Date','Close','signal']]

    sell = price_action.loc[price_action['signal']=='Sell']
    sell = sell[['Date','Close','signal']]
    
    fig = make_subplots(specs=[[{"secondary_y":True}]])        
    
    fig.add_trace(
        go.Candlestick(
            x = price_action['Date'],
            open = price_action['Open'],
            high = price_action['High'],
            low = price_action['Low'],
            close = price_action['Close'],
            name = 'stock',
            showlegend=True
        ),secondary_y=True
    )
    
    trace2 = {
        'x': price_action['Date'],
        'y': price_action['avg_30'],
        'type': 'scatter',
        'mode': 'lines',
        'line': {
            'width': 1,
            'color': 'orange'
                },
        'name': 'Moving Average of 30 periods'
    }
    
    fig.add_trace(trace2,secondary_y=True)
    

    
    trace3 = {
        'x': price_action['Date'],
        'y': price_action['upper_band'],
        'type': 'scatter',
        'mode': 'lines',
        'line': {'width': 1},
        'name': 'Bollinger Bands',
        'yaxis':'y',
        'marker':{'color':'indigo'},
        'hoverinfo':'none',
        'legendgroup':'Bollinger Bands'
    }

    trace4 = {
        'x': price_action['Date'],
        'y': price_action['lower_band'],
        'type': 'scatter',
        'mode': 'lines',
        'line': {'width': 1},
        'name': 'Bollinger Bands',
        'yaxis':'y',
        'marker':{'color':'indigo'},
        'hoverinfo':'none',
        'showlegend':False,
        'legendgroup':'Bollinger Bands'
    }
    
    fig.add_trace(trace3,secondary_y=True)
    fig.add_trace(trace4,secondary_y=True)
    
    fig.add_trace(go.Scatter(x=sell['Date'],y=sell['Close'],mode='markers',name='sell'),secondary_y=True)
    fig.add_trace(go.Scatter(x=buy['Date'],y=buy['Close'],mode='markers',name='buy'),secondary_y=True)
    
    for c_avg in support_resist:
        fig.add_hline(y = c_avg[0], line_width = 1, line_color = 'blue',secondary_y=True)
    
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(count=3, label="3y", step="year", stepmode="backward"),
                dict(count=5, label="5y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )   
    
    fig.update_layout(hovermode='x',title='Bollinger Band')
    fig.update_yaxes(title='Volume',secondary_y=False,showgrid=False)
    fig.update_yaxes(title='Price',secondary_y=True,showgrid=True)
    fig.show()
    
    
    
    # Combine Bollinger Band & KMeans Support Resistance
    signal_lst = ['Hold']*len(price_action)
    for i in range(len(price_action)):
        if price_action['Close'][i] < price_action['lower_band'][i]:
            closest_lvl = closest_support_resistance(price_action['Close'][i],support_resist)[0]
            if (price_action['High'][i] >= closest_lvl)|(price_action['Close'][i] >= closest_lvl):
                signal_lst[i] = 'Buy'
            elif (price_action['High'][i] >= closest_lvl)|(price_action['Close'][i] <= closest_lvl):
                signal_lst[i] = 'Sell'
        if price_action['Close'][i] > price_action['upper_band'][i]:
            closest_lvl = closest_support_resistance(price_action['Close'][i],support_resist)[0]
            if (price_action['High'][i] >= closest_lvl)|(price_action['Close'][i] >= closest_lvl):
                continue
            elif (price_action['High'][i] >= closest_lvl)|(price_action['Close'][i] <= closest_lvl):
                signal_lst[i] = 'Sell'
        else:
            continue

    price_action['cross_signal'] = signal_lst

    buy = price_action.loc[price_action['cross_signal']=='Buy']
    buy = buy[['Date','Close','cross_signal']]

    sell = price_action.loc[price_action['cross_signal']=='Sell']
    sell = sell[['Date','Close','cross_signal']]
    
    fig = make_subplots(specs=[[{"secondary_y":True}]])        
    
    fig.add_trace(
        go.Candlestick(
            x = price_action['Date'],
            open = price_action['Open'],
            high = price_action['High'],
            low = price_action['Low'],
            close = price_action['Close'],
            name = 'stock',
            showlegend=True
        ),secondary_y=True
    )
    
    trace2 = {
        'x': price_action['Date'],
        'y': price_action['avg_30'],
        'type': 'scatter',
        'mode': 'lines',
        'line': {
            'width': 1,
            'color': 'orange'
                },
        'name': 'Moving Average of 30 periods'
    }
    
    fig.add_trace(trace2,secondary_y=True)
    

    
    trace3 = {
        'x': price_action['Date'],
        'y': price_action['upper_band'],
        'type': 'scatter',
        'mode': 'lines',
        'line': {'width': 1},
        'name': 'Bollinger Bands',
        'yaxis':'y',
        'marker':{'color':'indigo'},
        'hoverinfo':'none',
        'legendgroup':'Bollinger Bands'
    }

    trace4 = {
        'x': price_action['Date'],
        'y': price_action['lower_band'],
        'type': 'scatter',
        'mode': 'lines',
        'line': {'width': 1},
        'name': 'Bollinger Bands',
        'yaxis':'y',
        'marker':{'color':'indigo'},
        'hoverinfo':'none',
        'showlegend':False,
        'legendgroup':'Bollinger Bands'
    }
    
    fig.add_trace(trace3,secondary_y=True)
    fig.add_trace(trace4,secondary_y=True)
    
    fig.add_trace(go.Scatter(x=sell['Date'],y=sell['Close'],mode='markers',name='sell'),secondary_y=True)
    fig.add_trace(go.Scatter(x=buy['Date'],y=buy['Close'],mode='markers',name='buy'),secondary_y=True)
    
    for c_avg in support_resist:
        fig.add_hline(y = c_avg[0], line_width = 1, line_color = 'blue',secondary_y=True)
    
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(count=3, label="3y", step="year", stepmode="backward"),
                dict(count=5, label="5y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )   
    
    fig.update_layout(hovermode='x',title='Bollinger Band + Support & Resistance')
    fig.update_yaxes(title='Volume',secondary_y=False,showgrid=False)
    fig.update_yaxes(title='Price',secondary_y=True,showgrid=True)
    fig.show()
    
    return support_resist,price_action

In [4]:
# Try to find the local maxima and local minima
# HK benchmark 3115.HK / 2800.HK
# US Benchmark SP500 -> IVV/ITOT/VOO/SPY, Nasdaq -> NASDX/QQQ, DOWS -> DIA, Russell 200 -> VTWO 
lst=['TSLA']
last5 = datetime.now() - timedelta(days=5*365+30)
last5_str = last5.strftime('%Y-%m-%d')
last_5 = data.DataReader(lst, start=last5_str).reset_index()

price = np.array(last_5['Adj Close']).reshape(-1,1)

support_resist,price_action = get_support_resist(price,last_5)

[*********************100%***********************]  1 of 1 completed


/Users/matthewong/opt/anaconda3/envs/finance/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [5]:
support_resist

[array([11.93133259]),
 array([77.60966492]),
 array([158.01000214]),
 array([211.16666412]),
 array([259.82832336]),
 array([320.81999207]),
 array([409.97000122])]

In [6]:
price_action.loc[price_action['cross_signal']=='Buy']

,Date,Open,High,Low,Close,Adj Close,Volume,upper_band,lower_band,avg_30,signal,cross_signal
71,2018-09-28,18.017332,18.533333,17.370667,17.651333,17.651333,504745500,21.994020,17.802246,19.898133,Buy,Buy
76,2018-10-05,18.309999,18.325333,17.333332,17.463333,17.463333,269167500,21.642508,17.588070,19.615289,Buy,Buy
77,2018-10-08,17.634666,17.850668,16.600000,16.704000,16.704000,202090500,21.615929,17.293493,19.454711,Buy,Buy
128,2018-12-20,21.803333,22.019333,20.791332,21.025333,21.025333,136078500,25.065099,21.491700,23.278400,Buy,Buy
130,2018-12-24,20.900000,20.966667,19.680000,19.692667,19.692667,83398500,25.403118,20.778571,23.090844,Buy,Buy
177,2019-03-05,18.799999,18.933332,18.006666,18.436001,18.436001,281470500,21.614272,18.777817,20.196044,Buy,Buy
178,2019-03-06,18.431999,18.767332,18.292667,18.416000,18.416000,155032500,21.703845,18.587444,20.145645,Buy,Buy
179,2019-03-07,18.589333,18.980000,18.283333,18.439333,18.439333,141637500,21.763310,18.479090,20.121200,Buy,Buy
213,2019-04-25,17.000000,17.266666,16.404667,16.508667,16.508667,327741000,19.397825,16.870930,18.134378,Buy,Buy
214,2019-04-26,16.433332,16.445333,15.408667,15.676000,15.676000,335410500,19.486020,16.539091,18.012555,Buy,Buy


In [7]:
avg_price = 0
position = 0
profit_loss = 0
for i in range(len(price_action)):
    if position == 0:
        if (price_action['cross_signal'][i] == 'Buy'):
            print(price_action['Date'][i], 'Buy')
            avg_price = (avg_price*position + price_action['Close'][i]*1000)/ (position+1000)
            position += 1000
            print('Average Cost:',avg_price,'Position',position)
        else:
            continue
    elif position > 0:
        if (price_action['cross_signal'][i] == 'Buy'):
            print(price_action['Date'][i], 'Buy')
            avg_price = (avg_price*position + price_action['Close'][i]*1000)/ (position+1000)
            position += 1000
            print('Average Cost:',avg_price,'Position',position)
        elif (price_action['cross_signal'][i] == 'Sell'):
            if avg_price > price_action['Close'][i]:
                continue
            else:
                print(price_action['Date'][i],'Sell')
                profit_loss += (price_action['Close'][i]- avg_price)*1000 
                position += -1000
                print('Average Cost:',avg_price,'Position',position,'P/L:',profit_loss)
        else:
            continue
    else:
        continue

2018-09-28 00:00:00 Buy
Average Cost: 17.65133285522461 Position 1000
2018-10-05 00:00:00 Buy
Average Cost: 17.55733299255371 Position 2000
2018-10-08 00:00:00 Buy
Average Cost: 17.27288881937663 Position 3000
2018-12-20 00:00:00 Buy
Average Cost: 18.210999965667725 Position 4000
2018-12-24 00:00:00 Buy
Average Cost: 18.50733337402344 Position 5000
2019-03-05 00:00:00 Buy
Average Cost: 18.495444615681965 Position 6000
2019-03-06 00:00:00 Buy
Average Cost: 18.484095437186106 Position 7000
2019-03-07 00:00:00 Buy
Average Cost: 18.47850012779236 Position 8000
2019-04-25 00:00:00 Buy
Average Cost: 18.25962977939182 Position 9000
2019-04-26 00:00:00 Buy
Average Cost: 18.001266765594483 Position 10000
2019-04-29 00:00:00 Buy
Average Cost: 17.82824247533625 Position 11000
2019-04-30 00:00:00 Buy
Average Cost: 17.66861120859782 Position 12000
2019-05-01 00:00:00 Buy
Average Cost: 17.509538577153133 Position 13000
2019-05-17 00:00:00 Buy
Average Cost: 17.263762065342494 Position 14000
2019-05-2

In [8]:
avg_price

166.32666755445075

In [9]:
position

4000

In [10]:
profit_loss

3678465.3263381044

In [11]:
unrealized = (price_action['Close'].iloc[-1] - avg_price)*position
print('Unrealized:',unrealized)

Unrealized: 496213.349313447


In [12]:
avg_price = 0
position = 0
profit_loss = 0
for i in range(len(price_action)):
    if position == 0:
        if (price_action['cross_signal'][i] == 'Buy'):
            print(price_action['Date'][i], 'Buy')
            avg_price = (avg_price*position + price_action['Close'][i]*1000)/ (position+1000)
            position += 1000
            print('Average Cost:',avg_price,'Position',position)
        else:
            continue
    elif position > 0:
        if (price_action['cross_signal'][i] == 'Buy'):
            print(price_action['Date'][i], 'Buy')
            avg_price = (avg_price*position + price_action['Close'][i]*1000)/ (position+1000)
            position += 1000
            print('Average Cost:',avg_price,'Position',position)
        elif (price_action['cross_signal'][i] == 'Sell'):
            print(price_action['Date'][i],'Sell')
            profit_loss += (price_action['Close'][i]- avg_price)*1000 
            position += -1000
            print('Average Cost:',avg_price,'Position',position,'P/L:',profit_loss)
        else:
            continue
    else:
        continue

2018-09-28 00:00:00 Buy
Average Cost: 17.65133285522461 Position 1000
2018-10-05 00:00:00 Buy
Average Cost: 17.55733299255371 Position 2000
2018-10-08 00:00:00 Buy
Average Cost: 17.27288881937663 Position 3000
2018-12-20 00:00:00 Buy
Average Cost: 18.210999965667725 Position 4000
2018-12-24 00:00:00 Buy
Average Cost: 18.50733337402344 Position 5000
2019-03-05 00:00:00 Buy
Average Cost: 18.495444615681965 Position 6000
2019-03-06 00:00:00 Buy
Average Cost: 18.484095437186106 Position 7000
2019-03-07 00:00:00 Buy
Average Cost: 18.47850012779236 Position 8000
2019-04-25 00:00:00 Buy
Average Cost: 18.25962977939182 Position 9000
2019-04-26 00:00:00 Buy
Average Cost: 18.001266765594483 Position 10000
2019-04-29 00:00:00 Buy
Average Cost: 17.82824247533625 Position 11000
2019-04-30 00:00:00 Buy
Average Cost: 17.66861120859782 Position 12000
2019-05-01 00:00:00 Buy
Average Cost: 17.509538577153133 Position 13000
2019-05-17 00:00:00 Buy
Average Cost: 17.263762065342494 Position 14000
2019-05-2

In [13]:
profit_loss

3364665.3394699106

In [14]:
unrealized = (price_action['Close'].iloc[-1] - avg_price)*position
unrealized

0.0